In [ ]:
#default_exp hybrid

In [2]:
#export

import os
import pickle
import attr

import pandas as pd
import scipy
import numpy as np

from game_recommender import steam_data, content_based, user_based

In [3]:
%cd ..

/home/kuba/Projects/game_recommender


In [4]:
game_dataset = steam_data.get_steam_ratings_dataset()

In [5]:
user_based_recommender = user_based.UserBasedRecommender.make_from_steam_ratings('log_hours')

In [6]:
content_based_recommender = content_based.ContentBasedRecommender.make_from_steam_metadata()

In [7]:
steam_df = steam_data.load_steam_df() 

In [8]:
chosen_games_substring = 'counter strike'
chosen_games_df = steam_data.get_games_by_name(steam_df, chosen_games_substring) 

In [9]:
ratings_df = pd.concat([game_dataset.data, game_dataset.target], axis=1)
item_user_pivoted_df = ratings_df.pivot_table(index='name', columns='user_id', values='log_hours')


i = 10
user_id = user_based_recommender.user_similarity_searcher.df.iloc[10,0]
user_ratings_raw = item_user_pivoted_df.iloc[:,10]
user_ratings = user_ratings_raw[~user_ratings_raw.isna()]

In [10]:
user_ratings

name
counter strike                     3.135494
counter strike global offensive    6.194405
counter strike nexon zombies       0.693147
counter strike source              0.693147
day of defeat                      0.693147
deathmatch classic                 0.693147
eldevin                            0.693147
half life                          0.693147
half life blue shift               0.693147
half life opposing force           0.693147
planetside 2                       0.693147
raceroom racing experience         0.336472
realm of the mad god               0.693147
ricochet                           0.693147
team fortress classic              0.693147
unturned                           0.693147
warface                            0.693147
Name: 547685, dtype: float64

In [11]:
content_based_recommender.recommend_similar_games(
    user_ratings,
    n_recommended=50,
    n_similar=10
)

planetside 2                       2.177436
world of tanks blitz mmo           2.022441
black squad                        1.820021
tanki online                       1.772372
blockade classic                   1.738157
heliborne                          1.709596
counter strike global offensive    1.639987
post scriptum                      0.996020
argo                               0.959608
squad                              0.955189
insurgency                         0.846770
supreme commander 2                0.795574
rise of liberty                    0.723758
ravenfield                         0.713363
day of defeat source               0.693147
fortress forever                   0.693147
starbreak                          0.693147
infestation the new z              0.693147
deathmatch classic                 0.693147
iracing                            0.693147
tom clancys rainbow six 3 gold     0.693147
allods online                      0.693147
arma 2 operation arrowhead      

In [12]:
user_based_recommender.recommend_games_from_similar_users(user_ratings=user_ratings, n_recommended=50, n_similar=10)

name
counter strike global offensive              5.144231
counter strike                               3.207789
call of duty 4 modern warfare                2.571812
counter strike nexon zombies                 1.436040
counter strike source                        1.070542
team fortress 2                              0.995118
call of duty black ops 2                     0.964861
tera                                         0.821691
dota 2                                       0.704395
team fortress classic                        0.657487
half life                                    0.657487
half life opposing force                     0.657487
half life blue shift                         0.657487
deathmatch classic                           0.641388
day of defeat                                0.641388
ricochet                                     0.641388
half life 2 deathmatch                       0.637845
just cause 2                                 0.633761
warface                

In [21]:
#export


@attr.s
class HybridRecommender:
    
    content_based_recommender: content_based.ContentBasedRecommender = attr.ib()
    user_based_recommender: user_based.UserBasedRecommender = attr.ib()

    def get_weighted_recommendations(
            self,
            user_ratings,
            n_recommended,
            user_recommendation_weight,
            content_recommendation_weight,
            mean_content_recommendation_weight,
            n_similar_users=10,
            n_similar_items=10):
        user_based_recommendations = user_recommendation_weight * self.user_based_recommender.recommend_games_from_similar_users(
            user_ratings=user_ratings,
            n_recommended=n_recommended,
            n_similar=n_similar_users
        )
        content_based_recommendations = content_recommendation_weight * self.content_based_recommender.recommend_similar_games(
            user_ratings=user_ratings,
            n_similar=n_similar_items,
            n_recommended=n_recommended
        )
        mean_content_based_recommendations = mean_content_recommendation_weight * self.content_based_recommender.recommend_mean_similar_games(
            user_ratings=user_ratings,
            n_similar=n_similar_items,
            n_recommended=n_recommended
        )
        all_recommendations = pd.concat([
            user_based_recommendations,
            content_based_recommendations,
            mean_content_based_recommendations
        ])
        deduplicated_recommendations = all_recommendations.groupby(all_recommendations.index).agg('mean')
        best_recommendations_indices = deduplicated_recommendations.argsort()[::-1]
        return deduplicated_recommendations[best_recommendations_indices[:n_recommended]]

In [22]:
recommender = HybridRecommender(content_based_recommender, user_based_recommender)

recommender.get_weighted_recommendations(user_ratings, 10, 1, 1, 1)

call of duty 4 modern warfare      2.571812
counter strike global offensive    2.442003
world of tanks blitz mmo           2.022441
counter strike                     1.857210
tanki online                       1.772372
blockade classic                   1.738157
heliborne                          1.709596
planetside 2                       1.367108
black squad                        1.161432
post scriptum                      0.996020
dtype: float64